In [ ]:
!pip install bitstring
import cv2
import numpy as np
from bitstring import BitArray
import multiprocessing

In [ ]:
!pip install ray
# from ray.util.multiprocessing import Pool
import ray
# !pip install dask
# !pip install 'fsspec>=0.3.3'
# import pandas as pd
# import dask.dataframe as dd
# from dask.multiprocessing import get

     |████████████████████████████████| 54.7 MB 139 kB/s 
     |████████████████████████████████| 119 kB 74.0 MB/s 


In [ ]:
import psutil

In [ ]:
def compress(uncompressed):
 
    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}
 
    w = ""
    result = []
    for c in uncompressed:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            dictionary[wc] = dict_size
            dict_size += 1
            w = c
 
    if w:
        result.append(dictionary[w])
    return result

def decompress(compressed):
    from io import StringIO
 
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}

    result = StringIO()
    w = chr(compressed.pop(0))
    result.write(w)
    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('error')
        result.write(entry)
 
        
        dictionary[dict_size] = w + entry[0]
        dict_size += 1
 
        w = entry
    return result.getvalue()
 
 

In [ ]:

@ray.remote
def compress_frame(a):
  uncompressed = "".join([chr(item) for item in a])
  compressed = compress(uncompressed)

  return BitArray().join(BitArray(uint=x, length=16) for x in compressed)

In [ ]:

@ray.remote
def decompress_frame(a):
  compressed = []
  
  for i in range(0,len(a),16):
    compressed.append(a[i:i+16].uint)
  decompressed = decompress(compressed)
  return decompressed

In [ ]:
# Opens the Video file
cap= cv2.VideoCapture("./video.mp4")

images = []
height = 0
width = 0
first = True
while(cap.isOpened()):
    ret, frame = cap.read()
    if first:
      print (frame.shape)
      height = len(frame)
      width = len(frame[0])
      first = False
  
    if ret == False:
        break
    usable = frame.flatten()
    a = usable.astype(np.uint8)
    images.append(a)

res = []

num_cpus = psutil.cpu_count()

print (num_cpus)

ray.shutdown()
ray.init(num_cpus=2, num_gpus=1)
compressed = ray.get([compress_frame.remote(i) for i in images[:5]])

import pickle

with open('todo.pickle', 'wb') as f:
  pickle.dump(compressed, f)

# for i in images[:100]:
#   res.append(compress_frame(i))

# print (res)
# pool = Pool()
# results = pool.map(compress_frame, images[0:100])
# ddata = dd.from_pandas(df, npartitions=2*multiprocessing.cpu_count())
# print (ddata)
# res = ddata.map_partitions(lambda df: df.apply((lambda row: compress_frame(*row)), axis=1)).compute(scheduler='threads')  
# print (res)

(270, 480, 3)
2


In [ ]:
with open('todo.pickle', 'rb') as ff:
  compressed = pickle.load(ff)
compressed

[BitArray('0x00000100010101020103010401050106010701080109010a010b010c010d010e010f0110011101120113011401150116011701180119011a011b011c011d011e011f0120012101220123012401250126012701280129012a012b012c012d012e012f0130013101320133013401350136013701380139013a013b013c01...') # length=679264,
 BitArray('0x00000100010101020103010401050106010701080109010a010b010c010d010e010f0110011101120113011401150116011701180119011a011b011c011d011e011f0120012101220123012401250126012701280129012a012b012c012d012e012f0130013101320133013401350136013701380139013a013b013c01...') # length=681760,
 BitArray('0x00000100010101020103010401050106010701080109010a010b010c010d010e010f0110011101120113011401150116011701180119011a011b011c011d011e011f0120012101220123012401250126012701280129012a012b012c012d012e012f0130013101320133013401350136013701380139013a013b013c01...') # length=681888,
 BitArray('0x00000100010101020103010401050106010701080109010a010b010c010d010e010f0110011101120113011401150116011701180119011a011b011c011d011e0

In [ ]:
NotebookApp.iopub_data_rate_limit = 1000000

In [ ]:
decompressed = ray.get([decompress_frame.remote(i) for i in compressed])
# decompressed = decompress_frame(compressed)

video_name = 'video.avi'

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

frames = []
for i in enumerate(decompressed):
  a = []
  for el in i[1]:
    a.append(ord(el))

  a = np.array(a).astype(np.uint8)
  reshaped = np.reshape(a, (height, width,3))

  cv2.imwrite("your_file"+ str(i[0]) +".jpg", reshaped)
  video.write(cv2.imread("your_file"+ str(i[0]) +".jpg"))

cv2.destroyAllWindows()
video.release()